In [2]:
import polars as pl

df = pl.read_parquet(
    "https://www.data.gouv.fr/fr/datasets/r/11cea8e8-df3e-4ed1-932b-781e2635e432"
)

# df = pl.read_parquet("/home/colin/git/decp-processing/dist/decp.parquet")

pl.Config(fmt_str_lengths=120, fmt_table_cell_list_len=50, set_tbl_rows=100)
print(pl.__version__)

NameError: name 'df' is not defined

In [6]:
df_uid = (
    df.select("uid", "montant", "donneesActuelles")
    .filter(pl.col("donneesActuelles") & (pl.col("montant") < 100000000))
    .unique("uid")
)

df_titu = df.select(
    "uid", "donneesActuelles", "titulaire_id", "titulaire_typeIdentifiant"
).filter(pl.col("donneesActuelles"))
print("somme", int(df_uid.select(pl.sum("montant")).item()))

print("total marchés uniques : ", df.unique("uid").height)
print("marchés/titulaires actuels (lignes) : ", df_titu.height)
print("total acheteurs uniques : ", df.unique("acheteur_id").height)
print("total titulaires uniques : ", df.unique("titulaire_id").height)


for year in range(2018, 2025 + 1):
    year = str(year)
    print("# " + year)
    print(
        "  - marches :".ljust(20, " "),
        df.filter(pl.col("dateNotification").dt.year() == int(year))
        .unique("uid")
        .height,
    )
    print(
        "  - acheteurs :".ljust(20, " "),
        df.filter(pl.col("dateNotification").dt.year() == int(year))
        .unique("acheteur_id")
        .height,
    )
    print(
        "  - tiul :".ljust(20, " "),
        df.filter(pl.col("dateNotification").dt.year() == int(year))
        .unique("titulaire_id")
        .height,
    )

somme 1776566
total marchés uniques :  9
marchés/titulaires actuels (lignes) :  11
total acheteurs uniques :  6
total titulaires uniques :  17
# 2018
  - marches :        0
  - acheteurs :      0
  - tiul :           0
# 2019
  - marches :        1
  - acheteurs :      1
  - tiul :           2
# 2020
  - marches :        2
  - acheteurs :      2
  - tiul :           2
# 2021
  - marches :        1
  - acheteurs :      1
  - tiul :           1
# 2022
  - marches :        5
  - acheteurs :      3
  - tiul :           5
# 2023
  - marches :        5
  - acheteurs :      4
  - tiul :           6
# 2024
  - marches :        1
  - acheteurs :      1
  - tiul :           1
# 2025
  - marches :        5
  - acheteurs :      4
  - tiul :           6


In [7]:
print(
    df.select("uid", "sourceDataset")
    .unique()
    .group_by("sourceDataset")
    .len()
    .sort("len", descending=True)
)

# df = df.filter(pl.col("datePublicationDonnees").cast(pl.String) > "2018").with_columns(
#     (pl.col("datePublicationDonnees").dt.year().cast(pl.String) + "_" +
#      pl.col("datePublicationDonnees").dt.month().cast(pl.String).str.pad_start(2, "0"))
#     .alias("moisPublicationDonnees"))
# print("aws par mois", df.filter(pl.col("sourceDataset") == "datagouv_aws").
#       select("uid", "moisPublicationDonnees").unique("uid")
#       .group_by("moisPublicationDonnees").len()
#      .sort(by="moisPublicationDonnees"))

shape: (1, 2)
┌───────────────┬─────┐
│ sourceDataset ┆ len │
│ ---           ┆ --- │
│ str           ┆ u32 │
╞═══════════════╪═════╡
│ test_dataset  ┆ 9   │
└───────────────┴─────┘


In [8]:
pl.scan_parquet(
    "data/resource_cache/1e83804b1e792a849e50d3fe5f5de2b449ff3e9a.parquet"
).collect()

uid,dateNotification,datePublicationDonnees,montant,dureeMois,titulaire_id,titulaire_typeIdentifiant,procedure,nature,codeCPV,id,formePrix,objet,sourceDataset,lieuExecution_code,lieuExecution_typeCode,marcheInnovant,attributionAvance,sousTraitanceDeclaree,ccag,offresRecues,typeGroupementOperateurs,idAccordCadre,tauxAvance,origineUE,origineFrance,acheteur_id,considerationsSociales,considerationsEnvironnementales,modalitesExecution,techniques,typesPrix,sourceFile
str,date,date,f64,i16,str,str,str,str,str,str,str,str,str,str,str,bool,bool,bool,str,i16,str,str,f32,f32,f32,str,str,str,str,str,str,str
